1/ Objectif 
Vous devrez réaliser un modèle de classification de texte à l'aide d'un réseau de neurones récurent (RNN), sans utiliser de module préfabriqué (nn.RNN). Le modèle devra prédire un label (ex. positif, négatif ou bien neutre) en fonction de votre jeux de donnée. Attention vous aurez besoin d'un jeux de donnée permettant un apprentissage supervisé.
 
2/ Etape dans la construction du modèle 
Prétraitement des données : vous avez déjà vu cette partie, je n'ai pas besoin de vous la redétailler ; 
Architecture du modèle : vous devrez implémenter un RNN "from scratch" 
Une couche d'embedding (nn.Embedding) pour transformer les indices en vecteurs. 
Un bloc RNN, ou l'état caché (hidden_state) est mis à jour de façon récursive (vous avez la formule pour rappel dans le ppt). 
Une couche linéaire finale pour produire votre prédiction à partir du dernier état caché. 
Optionnel (si tout fonctionne) : dropout, batchnorm, etc.. 

3/ Entraînement 
Attention, il va falloir modifier la fonction de perte habituel (MSELoss) pour ce cas d'application ! Suivez les métriques de précision sur le jeu de validation (ou de test). Afficher les métriques toutes les X époques. Enfin, je conseille d'afficher un graphique avec Matplotlib pour vérifier l'entraînement, nottament, la courbe d'apprentissage vs la courbe de validation (ou test). 
 
4/ Mes conseils
Commencez simple. Une seule couche RNN suffit.
Testez d'abord sur un petit sous-ensemble pour valider le code.
Gardez une longueur fixe pour les séquences (padding).
Vérifiez vos dimensions avec .shape à chaque étape.

# ============================
# Imports et préparations
# ============================

In [1]:
!pip install kagglehub

In [2]:
import pandas as pd

import kagglehub
import os
import re

/Users/ericcosterousse/.pyenv/versions/3.11.0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ============================
# 1. Chargement et nettoyage
# ============================

In [3]:
# Download latest version
path = kagglehub.dataset_download("abdelmalekeladjelet/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/ericcosterousse/.cache/kagglehub/datasets/abdelmalekeladjelet/sentiment-analysis-dataset/versions/1


In [4]:
# Liste les fichiers dans ce dossier
print("Contenu du dossier:")
print(os.listdir(path))

Contenu du dossier:
['sentiment_data.csv']


In [5]:
df = pd.read_csv(os.path.join(path, 'sentiment_data.csv'))

In [6]:
df.head()

,Unnamed: 0,Comment,Sentiment
0,0,lets forget apple pay required brand new iphon...,1
1,1,nz retailers don’t even contactless credit car...,0
2,2,forever acknowledge channel help lessons ideas...,2
3,3,whenever go place doesn’t take apple pay doesn...,0
4,4,apple pay convenient secure easy use used kore...,2


In [7]:
df.shape

(241145, 3)

In [8]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df.head()

,Comment,Sentiment
0,lets forget apple pay required brand new iphon...,1
1,nz retailers don’t even contactless credit car...,0
2,forever acknowledge channel help lessons ideas...,2
3,whenever go place doesn’t take apple pay doesn...,0
4,apple pay convenient secure easy use used kore...,2


# Pourquoi prétraiter les données ?
Le texte brut n’est pas directement exploitable par un réseau de neurones. Il faut le transformer en une forme numérique compréhensible par le modèle, tout en gardant le maximum d’information.

### 1. Nettoyage du texte (optionnel mais recommandé)
Objectif : enlever le bruit (URLs, ponctuations inutiles, mentions @, hashtags non pertinents, caractères spéciaux, etc.)

Pourquoi ? Ces éléments peuvent nuire à la qualité des embeddings et du modèle.

In [10]:
import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

class Preprocessing:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stop_words.update(['u', 'us', 'q'])
        self.lemmatizer = WordNetLemmatizer()

    def clean_text(self, text):
        """
        Nettoyage du texte : minuscules, suppression HTML, ponctuation, chiffres, espaces multiples.
        """
        if not isinstance(text, str):
            return ""

        text = text.lower()
        # Normalisation unicode (accents etc.)
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
        # Supprimer les URLs (http, https, www)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
        # Supprimer les emails
        text = re.sub(r'\S+@\S+', '', text)
    
        # Supprimer ponctuation et chiffres, garder lettres et espaces uniquement
        # (on enlève aussi les acronymes avec points en une fois)
        text = re.sub(r'\b[a-z]\.', '', text)  # enlever lettres suivies d'un point (ex: u.)
        text = re.sub(r'[^a-z\s]', ' ', text)  # garder lettres et espaces uniquement
    
        # Enlever espaces multiples
        text = re.sub(r'\s+', ' ', text).strip()
    
        return text
    
    def tokenize(self, text):
        """
        Tokenisation simple par split des mots.
        """
        return text.split()
    
    def remove_stopwords(self, tokens):
        """
        Suppression des stopwords.
        """
        filtered = [token for token in tokens if token not in self.stop_words]  
        return filtered
    
    def lemmatize(self, tokens):
        """
        Lemmatisation des tokens.
        """
        return [self.lemmatizer.lemmatize(token) for token in tokens]
    
    def preprocess(self, text):
        """
        Pipeline complet combinant toutes les étapes
        """
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        no_stop = self.remove_stopwords(tokens)
        lemmas = self.lemmatize(no_stop)
        return " ".join(lemmas)

In [11]:
prep = Preprocessing()
tweet = "I love the new policy from the government! Check http://example.com"
processed = prep.preprocess(tweet)
print(processed)
# Résultat possible : "love new policy check"

love new policy government check


In [12]:
preprocess = Preprocessing()
df['Cleaned_Comment'] = df['Comment'].astype(str).apply(preprocess.preprocess)

In [13]:
df.head()

,Comment,Sentiment,Cleaned_Comment
0,lets forget apple pay required brand new iphon...,1,let forget apple pay required brand new iphone...
1,nz retailers don’t even contactless credit car...,0,nz retailer dont even contactless credit card ...
2,forever acknowledge channel help lessons ideas...,2,forever acknowledge channel help lesson idea e...
3,whenever go place doesn’t take apple pay doesn...,0,whenever go place doesnt take apple pay doesnt...
4,apple pay convenient secure easy use used kore...,2,apple pay convenient secure easy use used kore...


In [14]:
from collections import Counter

# Chaque ligne est déjà un string "token1 token2 token3 ..."
tokenized_texts = [text.split() for text in df['Cleaned_Comment']]

# Construire le vocabulaire (compter les mots)
word_counts = Counter()
for tokens in tokenized_texts:
    word_counts.update(tokens)

# Limiter la taille du vocabulaire (tu peux ajuster)
vocab_size = 5000

most_common = word_counts.most_common(vocab_size - 2)  # -2 pour <PAD> et <UNK>

# Mapping mot → index
word2idx = {'<PAD>': 0, '<UNK>': 1}
for i, (word, _) in enumerate(most_common, start=2):
    word2idx[word] = i

In [15]:
max_len = 50  # longueur fixe de séquence

def encode_and_pad(tokens):
    encoded = [word2idx.get(token, word2idx['<UNK>']) for token in tokens]
    if len(encoded) > max_len:
        return encoded[:max_len]
    else:
        return encoded + [word2idx['<PAD>']] * (max_len - len(encoded))

encoded_texts = [encode_and_pad(text.split()) for text in df['Cleaned_Comment']]

In [16]:
labels = df['Sentiment'].values  # 0,1,2 

In [17]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, encoded_texts, labels):
        self.encoded_texts = encoded_texts
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.encoded_texts[idx], dtype=torch.long)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ericcosterousse/.pyenv/versions/3.11.0/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/ericcosterousse/.pyenv/versions/3.11.0/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/ericcosterousse/.pyenv/versions/3.11.0/lib/python3.11/site-packages/ipykernel/kernelapp

In [18]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

X_train, X_test, y_train, y_test = train_test_split(encoded_texts, labels, test_size=0.2, random_state=42)

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# ============================
# Embeddings, Normalisation & RNN
# ============================

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNFromScratch(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.layer_norm = nn.LayerNorm(embedding_dim)
        
        # Matrices de poids
        self.W_ih = nn.Linear(embedding_dim, hidden_dim, bias=True)
        self.W_hh = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
        self.output_layer = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        """
        x: LongTensor (batch_size, seq_len)
        """
        batch_size, seq_len = x.shape
        
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        embedded = self.layer_norm(embedded)
        
        # Initialisation état caché h_0 = 0
        h_t = torch.zeros(batch_size, self.W_hh.out_features, device=x.device)
        
        # Boucle temporelle
        for t in range(seq_len):
            x_t = embedded[:, t, :]  # (batch_size, embedding_dim)
            h_t = torch.tanh(self.W_ih(x_t) + self.W_hh(h_t))
        
        # Prédiction à partir du dernier état caché
        out = self.output_layer(h_t)  # (batch_size, output_dim)
        
        return out



In [20]:
import torch.optim as optim

model = RNNFromScratch(vocab_size=5000, embedding_dim=100, hidden_dim=64, output_dim=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [21]:
import torch.optim as optim
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RNNFromScratch(vocab_size, embedding_dim=100, hidden_dim=64, output_dim=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_loss = total_loss / total
    train_acc = correct / total
    print(f"Epoch {epoch+1} | Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")


Epoch 1 | Loss: 1.0682 | Accuracy: 0.4270
Epoch 2 | Loss: 1.0677 | Accuracy: 0.4276
Epoch 3 | Loss: 1.0676 | Accuracy: 0.4276
Epoch 4 | Loss: 1.0675 | Accuracy: 0.4277
Epoch 5 | Loss: 1.0675 | Accuracy: 0.4278
Epoch 6 | Loss: 1.0674 | Accuracy: 0.4278
Epoch 7 | Loss: 1.0674 | Accuracy: 0.4279
Epoch 8 | Loss: 1.0674 | Accuracy: 0.4279
Epoch 9 | Loss: 1.0674 | Accuracy: 0.4279
Epoch 10 | Loss: 1.0673 | Accuracy: 0.4279


In [22]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f"Test Accuracy: {correct / total:.4f}")


Test Accuracy: 0.4284
